## Sentiment Analysis

Use the newsapi to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
    1.Which coin had the highest mean positive score?
    2. Which coin had the highest negative score?
    3. Which coin had the highest positive score?

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
load_dotenv()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jpmdi\AppData\Roaming\nltk_data...


In [3]:
api_key =("34d0473c17c24afcb1997aad110bb7cc")

In [4]:
#Create a newsapi client

from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key = api_key)

ModuleNotFoundError: No module named 'newsapi'

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_articles = newsapi.get_everything(q="Bitcoin", language="en", sort_by="relevancy")


In [ ]:
# Fetch the Ethereum news articles

ethereum_articles = newsapi.get_everything(q='ethereum', language='en')


In [ ]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []

for article in bitcoin_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_df = pd.DataFrame(bitcoin_sentiments)
bitcoin_df.head()

In [ ]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_df = pd.DataFrame(ethereum_sentiments)
ethereum_df.head()

In [ ]:
# Describe the Bitcoin Sentiment

bitcoin_df.describe()

In [ ]:
# Describe the Ethereum Sentiment

ethereum_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin has the highest mean positive score

Q: Which coin had the highest compound score?

A: Bitcoin has the highest compound score

Q. Which coin had the highest positive score?

A: Bitcoin has the highest positive score

### Natural Language Processing

### Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:

    1. Lowercase each word.
    2. Remove Punctuation.
    3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
#Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
#Create a list of stopwords
sw = set(stopwords.words('english'))

In [ ]:
#Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    #Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('',text)
    
    #Create a tokenized list of the words
    words = word_tokenize(text)
    
    #Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    
    #Convert the words to lowercase
    re_words = word_tokenize(re_clean)
    
    #Remove the stop words
    re_result = [word.lower() for word in re_words if word.lower() not in sw]
    
    return re_result

In [ ]:
#create a new tokens column for Bitcoin
bitcoin_df['tokens'] = bitcoin_df['text'].apply(tokenizer)
bitcoin_df.head()

In [ ]:
#Create a new tokens column for Ethereum
ethereum_df['tokens'] = ethereum_df['text'].apply(tokenizer)
ethereum_df.head()

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin.

1. Use NLTK to produce the n-grams for N = 2
2. List the top 10 words for each coin.

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
#Generate the Bitcoin N-grams where N=2
bitcoin_a = tokenizer(bitcoin_df.text.str.cat())
bitcoin_counts = Counter(ngrams(bitcoin_a,n=2))
print(dict(bitcoin_counts))

In [ ]:
#Generate the ethereum N-grams where N=2
ethereum_a = tokenizer(ethereum_df.text.str.cat())
ethereum_counts= Counter(ngrams(ethereum_a,n=2))
print(dict(ethereum_counts))

In [ ]:
#Function token_count generates the top 10 words for a given coin
def token_count(tokens,N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
#Use token_count to get the top 10 words for Bitcoin
bitcoin_top_10 = token_count(bitcoin_a,10)
bitcoin_top_10

In [ ]:
#Use token_count to get the top 10 words for Ethereum
ethereum_top_10 = token_count(ethereum_a,10)
ethereum_top_10

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize']=[20.0,10.0]

In [ ]:
#Generate the Bitcoin word cloud
wc = WordCloud().generate(bitcoin_df.text.str.cat())
plt.imshow(wc)

In [ ]:
#Generate the Ethereum word cloud
wc = WordCloud().generate(ethereum_df.text.str.cat())
plt.imshow(wc)

## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
#Download the language model for spacy
!python -m spacy download en_core_web_sm

In [ ]:
#Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
#Concentrate all the Bitcoin text together
concat_bitcoin = bitcoin_df.text.str.cat()
concat_bitcoin

In [ ]:
#Run the NER processor on all of the text
bitcoin_doc = nlp(concat_bitcoin)
bitcoin_doc

#Add a title to the document
bitcoin_doc.user_data['Title'] = 'Bitcoin NER'

In [ ]:
#Render the visualization
displacy.render(bitcoin_doc, style = 'ent')

In [ ]:
#list all Entities
for i in bitcoin_doc.ents:
    print(i.text, i.label_)

### Ethereum NER

In [ ]:
#Concatenate all of the Ethereum text together
concat_ethereum = ethereum_df.text.str.cat()
concat_ethereum

In [ ]:
#Run the NER processor on all of the text
ethereum_doc = nlp(concat_ethereum)
ethereum_doc

#Add a title to the document
ethereum_doc.user_data['Title'] = 'Ethereum NER'

In [ ]:
#Render the visualization
displacy.render(ethereum_doc, style = 'ent')

In [ ]:
#List all Entities
for i in ethereum_doc.ents:
    print(i.text, i.label_)